# Named Entity Recognition

## Set variables

In [1]:
file_name = "BiodivNER_ELECTRA.ipynb"
batch_size = 4
dataset = "BiodivNER"
data_directory = "../Datasets/NER/BiodivNER/"
model_src = "google/electra-base-discriminator"

## Install and import necessary libraries

In [2]:
# !pip install transformers
# !pip install datasets
# !pip install seqeval
# !pip install accelerate -U

In [3]:
import os
import shutil
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Convert the dataset

In [13]:
root_data_dir = data_directory

biodivner_dataset = "train"
train_csv_file_path = "train.csv"
val_csv_file_path = "dev.csv"
test_csv_file_path = "test.csv"

In [14]:
def loadData(csv_file_path):
  dataset_path = os.path.join(root_data_dir, csv_file_path)
  data = pd.read_csv(dataset_path, encoding="latin1")
  data = data.fillna(method="ffill")
  return data

In [15]:
data = loadData(train_csv_file_path)
val_data = loadData(val_csv_file_path)
test_data = loadData(test_csv_file_path)

/tmp/ipykernel_1587958/3117403119.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")
/tmp/ipykernel_1587958/3117403119.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")
/tmp/ipykernel_1587958/3117403119.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")


In [17]:
import re

def convert(orig):

    df = pd.DataFrame(orig)

    # Extract the integer from 'Sentence #'
    df['Sentence #'] = df['Sentence #'].apply(lambda x: int(re.search(r'\d+', x).group()))

    # Add a column representing the original order
    df['Original Order'] = range(len(df))

    # Group by 'Sentence #' and aggregate 'Word' and 'Tag' into lists
    grouped = df.groupby('Sentence #').agg({'Word': list, 'Tag': list, 'Original Order': 'first'}).reset_index()

    # Sort the DataFrame based on the original order
    grouped = grouped.sort_values(by='Original Order').drop('Original Order', axis=1)

    grouped = grouped.rename(columns={'Word': 'tokens'})
    grouped = grouped.rename(columns={'Tag': 'labels'})
    grouped = grouped.drop('Sentence #', axis=1)
    # print(grouped)
    return grouped

# print(grouped)


In [18]:
train_df = convert(data)
val_df = convert(val_data)
test_df = convert(test_data)

In [20]:
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

## Tokenize the dataset

In [22]:
label_list = ['B-Phenomena', 'I-Phenomena', 'B-Quality', 'I-Quality', 'B-Location', 'I-Location', 'B-Environment', 'I-Environment', 'B-Matter', 'I-Matter', 'B-Organism', 'I-Organism', 'O']
label2id = {k: v for v, k in enumerate(label_list)}
id2label = {v: k for v, k in enumerate(label_list)}
print(label2id)
print(id2label)

{'B-Phenomena': 0, 'I-Phenomena': 1, 'B-Quality': 2, 'I-Quality': 3, 'B-Location': 4, 'I-Location': 5, 'B-Environment': 6, 'I-Environment': 7, 'B-Matter': 8, 'I-Matter': 9, 'B-Organism': 10, 'I-Organism': 11, 'O': 12}
{0: 'B-Phenomena', 1: 'I-Phenomena', 2: 'B-Quality', 3: 'I-Quality', 4: 'B-Location', 5: 'I-Location', 6: 'B-Environment', 7: 'I-Environment', 8: 'B-Matter', 9: 'I-Matter', 10: 'B-Organism', 11: 'I-Organism', 12: 'O'}


In [23]:
task = "ner"
model_checkpoint = model_src

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json: 100%|██████████| 27.0/27.0 [00:00<00:00, 41.4kB/s]
config.json: 100%|██████████| 666/666 [00:00<00:00, 634kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 570kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 795kB/s]


In [24]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), max_length= 512, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
dev_tokenized_datasets = dev_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 240/240 [00:00<00:00, 1178.59 examples/s]


## Finetuning the model to the dataset

In [26]:
model =  AutoModelForTokenClassification.from_pretrained(model_checkpoint,id2label=id2label, label2id=label2id)

args = TrainingArguments(
    f"test-{task}",
    per_device_train_batch_size=batch_size,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    num_train_epochs=3
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=dev_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

pytorch_model.bin: 100%|██████████| 440M/440M [00:12<00:00, 34.2MB/s] 
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1587958/547268042.py:16: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


Clear model directory if it hasn't been cleared yet.

In [27]:
directory_path = "./ner.model"

if os.path.exists(directory_path):
    try:
        # Remove the directory
        shutil.rmtree(directory_path)
        print(f"Directory '{directory_path}' removed successfully.")
    except Exception as e:
        print(f"Error removing directory '{directory_path}': {e}")
else:
    print(f"Directory '{directory_path}' does not exist.")

Directory './ner.model' removed successfully.


In [28]:
trainer.train()
trainer.evaluate()
trainer.save_model('ner.model')

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.362200,0.275987,0.506832,0.368231,0.426555,0.927394
2,0.167400,0.176464,0.578860,0.632671,0.604571,0.941596
3,0.104800,0.133625,0.676277,0.740975,0.707149,0.954690


/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


## Get precision, f1-score, and recall for each entity group

In [33]:
predictions, labels, _ = trainer.predict(test_tokenized_datasets)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'Environment': {'precision': 0.6153846153846154,
  'recall': 0.7120418848167539,
  'f1': 0.6601941747572816,
  'number': 191},
 'Location': {'precision': 0.3137254901960784,
  'recall': 0.5161290322580645,
  'f1': 0.3902439024390244,
  'number': 62},
 'Matter': {'precision': 0.6901408450704225,
  'recall': 0.4666666666666667,
  'f1': 0.5568181818181818,
  'number': 210},
 'Organism': {'precision': 0.7325038880248833,
  'recall': 0.8658088235294118,
  'f1': 0.7935973041280541,
  'number': 544},
 'Phenomena': {'precision': 0.704225352112676,
  'recall': 0.625,
  'f1': 0.662251655629139,
  'number': 80},
 'Quality': {'precision': 0.7223230490018149,
  'recall': 0.804040404040404,
  'f1': 0.7609942638623328,
  'number': 495},
 'overall_precision': 0.684971098265896,
 'overall_recall': 0.7490518331226296,
 'overall_f1': 0.7155797101449275,
 'overall_accuracy': 0.9359037849310223}